In [43]:
import os
from PIL import Image
import numpy as np

INPUT_FOLDER = "images/original"
SEPIA_FOLDER = "images/sepia"
NORMAL_FOLDER = "images/normal"

os.makedirs(SEPIA_FOLDER, exist_ok=True)
os.makedirs(NORMAL_FOLDER, exist_ok=True)

def apply_sepia(image):
    sepia_filter = np.array([[0.393, 0.769, 0.189],
                             [0.349, 0.686, 0.168],
                             [0.272, 0.534, 0.131]])
    img = np.array(image)
    sepia_img = img @ sepia_filter.T
    sepia_img = np.clip(sepia_img, 0, 255).astype(np.uint8)
    return Image.fromarray(sepia_img)

#itereaza peste imagini
for filename in os.listdir(INPUT_FOLDER):
    image_path = os.path.join(INPUT_FOLDER, filename)
    image = Image.open(image_path).convert("RGB").resize((64, 64))  #uniformzare
    #originalul
    image.save(os.path.join(NORMAL_FOLDER, filename))
    #varianta cu sepia
    sepia_image = apply_sepia(image)
    sepia_image.save(os.path.join(SEPIA_FOLDER, filename))


In [108]:
def load_dataset(normal_path="images/normal", sepia_path="images/sepia"):
    X = []
    y = []

    #imagini normale (eticheta 0)
    for file in os.listdir(normal_path):
        img = Image.open(os.path.join(normal_path, file)).convert("RGB")
        X.append(np.array(img).flatten() / 255.0)  # scalare 0-1
        y.append(0)

    #imagini sepia (eticheta 1)
    for file in os.listdir(sepia_path):
        img = Image.open(os.path.join(sepia_path, file)).convert("RGB")
        X.append(np.array(img).flatten() / 255.0)
        y.append(1)

    return np.array(X), np.array(y)

ANN TOOL

In [120]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

# Parametri
IMG_SIZE = (64, 64)
normal_path = "images/normal"
sepia_path = "images/sepia"

def load_images_from_folder(folder, label):
    data = []
    for filename in os.listdir(folder):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(folder, filename)
            img = Image.open(img_path).convert("RGB")
            img = img.resize(IMG_SIZE)
            img_array = np.array(img) / 255.0  # normalizare
            data.append((img_array, label))
    return data

normal_data = load_images_from_folder(normal_path, 0)
sepia_data = load_images_from_folder(sepia_path, 1)

all_data = normal_data + sepia_data
np.random.shuffle(all_data)

#imagini si etichete
X = np.array([item[0] for item in all_data])
y = np.array([item[1] for item in all_data])

print("X shape:", X.shape)
print("y unique values:", np.unique(y))

#impartire date
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, shuffle=True, random_state=42
)

print("Train label distribution:", np.unique(y_train, return_counts=True))
print("Test label distribution:", np.unique(y_test, return_counts=True))

# Aplatizare imagini
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Definire si antrenare model
model = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    activation='relu',
    solver='adam',
    learning_rate_init=0.001,
    max_iter=500,
    random_state=42,
    verbose=True
)

model.fit(X_train_flat, y_train)

# Predictii si evaluare
y_pred = model.predict(X_test_flat)
print("Distribuție predictii:", np.unique(y_pred, return_counts=True))

print("\nClasificare pe setul de test:")
print(classification_report(y_test, y_pred))
print(f"Acuratete: {accuracy_score(y_test, y_pred):.4f}")


X shape: (40, 64, 64, 3)
y unique values: [0 1]
Train label distribution: (array([0, 1]), array([16, 16]))
Test label distribution: (array([0, 1]), array([4, 4]))
Iteration 1, loss = 0.71302696
Iteration 2, loss = 3.36285012
Iteration 3, loss = 4.66495313
Iteration 4, loss = 2.84783949
Iteration 5, loss = 0.79543790
Iteration 6, loss = 1.83837648
Iteration 7, loss = 1.11391227
Iteration 8, loss = 1.04124553
Iteration 9, loss = 1.32096462
Iteration 10, loss = 0.67506927
Iteration 11, loss = 0.93623411
Iteration 12, loss = 1.00352733
Iteration 13, loss = 0.64122508
Iteration 14, loss = 0.78524071
Iteration 15, loss = 0.90624961
Iteration 16, loss = 0.64522137
Iteration 17, loss = 0.66261643
Iteration 18, loss = 0.80049392
Iteration 19, loss = 0.64598132
Iteration 20, loss = 0.59104821
Iteration 21, loss = 0.72126857
Iteration 22, loss = 0.61846431
Iteration 23, loss = 0.56205451
Iteration 24, loss = 0.64950168
Iteration 25, loss = 0.57638154
Iteration 26, loss = 0.54092410
Iteration 27, 

In [72]:
import numpy as np

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_deriv(z):
    s = sigmoid(z)
    return s * (1 - s)

class SimpleANN:
    def __init__(self, input_size, hidden_size=64, learning_rate=0.01):
        self.lr = learning_rate
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, 1) * 0.01
        self.b2 = np.zeros((1, 1))

    def forward(self, X):
        self.Z1 = X @ self.W1 + self.b1
        self.A1 = sigmoid(self.Z1)
        self.Z2 = self.A1 @ self.W2 + self.b2
        self.A2 = sigmoid(self.Z2)
        return self.A2

    def backward(self, X, y):
        m = y.shape[0]
        # Reshape y ca sa fie dimensiuni corecte pt extragere
        dZ2 = self.A2 - y.reshape(-1, 1)
        dW2 = self.A1.T @ dZ2 / m
        db2 = np.sum(dZ2, axis=0, keepdims=True) / m

        dA1 = dZ2 @ self.W2.T
        dZ1 = dA1 * sigmoid_deriv(self.Z1)
        dW1 = X.T @ dZ1 / m
        db1 = np.sum(dZ1, axis=0, keepdims=True) / m

        # Actualizare
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1

    def train(self, X, y, epochs=100):
        for i in range(epochs):
            y_hat = self.forward(X)
            self.backward(X, y)
            if i % 10 == 0:
                loss = -np.mean(y * np.log(y_hat + 1e-8) + (1 - y) * np.log(1 - y_hat + 1e-8))
                print(f"Epoch {i} - Loss: {loss:.4f}")

    def predict(self, X):
        return (self.forward(X) > 0.5).astype(int)

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = load_dataset("images/normal", "images/sepia")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = SimpleANN(input_size=X.shape[1], hidden_size=64, learning_rate=0.1)
model.train(X_train, y_train, epochs=100)

y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test accuracy:", acc)


Epoch 0 - Loss: 0.6929
Epoch 10 - Loss: 0.6913
Epoch 20 - Loss: 0.6915
Epoch 30 - Loss: 0.6920
Epoch 40 - Loss: 0.6930
Epoch 50 - Loss: 0.6947
Epoch 60 - Loss: 0.6981
Epoch 70 - Loss: 0.7052
Epoch 80 - Loss: 0.7195
Epoch 90 - Loss: 0.7448
Test accuracy: 0.0


In [109]:
import numpy as np

# functii de activar
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_deriv(z):
    s = sigmoid(z)
    return s * (1 - s)

def relu(z):
    return np.maximum(0, z)

def relu_deriv(z):
    return (z > 0).astype(float)

class SimpleANN:
    def __init__(self, input_size, hidden_size=64, learning_rate=0.01):
        self.lr = learning_rate
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, 1) * 0.01
        self.b2 = np.zeros((1, 1))

    def forward(self, X):
        self.Z1 = X @ self.W1 + self.b1
        self.A1 = relu(self.Z1)
        self.Z2 = self.A1 @ self.W2 + self.b2
        self.A2 = sigmoid(self.Z2)  # strat ieșire cu sigmoid
        return self.A2

    def backward(self, X, y):
        m = y.shape[0]
        dZ2 = self.A2 - y.reshape(-1, 1)
        dW2 = self.A1.T @ dZ2 / m
        db2 = np.sum(dZ2, axis=0, keepdims=True) / m

        dA1 = dZ2 @ self.W2.T
        dZ1 = dA1 * relu_deriv(self.Z1)
        dW1 = X.T @ dZ1 / m
        db1 = np.sum(dZ1, axis=0, keepdims=True) / m

        # actualizare
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1

    def train(self, X, y, epochs=100):
        for i in range(epochs):
            y_hat = self.forward(X)
            self.backward(X, y)
            if i % 10 == 0:
                loss = -np.mean(y * np.log(y_hat + 1e-8) + (1 - y) * np.log(1 - y_hat + 1e-8))
                print(f"Epoch {i} - Loss: {loss:.4f}")

    def predict(self, X):
        return (self.forward(X) > 0.5).astype(int)


In [110]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = load_dataset("images/normal", "images/sepia")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = SimpleANN(input_size=X.shape[1], hidden_size=64, learning_rate=0.1)
model.train(X_train, y_train, epochs=100)

y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test accuracy:", acc)


Epoch 0 - Loss: 0.6944
Epoch 10 - Loss: 0.6951
Epoch 20 - Loss: 0.7022
Epoch 30 - Loss: 0.7012
Epoch 40 - Loss: 0.6938
Epoch 50 - Loss: 0.6937
Epoch 60 - Loss: 0.7296
Epoch 70 - Loss: 0.6998
Epoch 80 - Loss: 0.6964
Epoch 90 - Loss: 0.6959
Test accuracy: 0.625


In [112]:
model.train(X_train, y_train, epochs=500)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test accuracy:", acc)



Epoch 0 - Loss: 0.7645
Epoch 10 - Loss: 0.7104
Epoch 20 - Loss: 0.7101
Epoch 30 - Loss: 0.7119
Epoch 40 - Loss: 0.7178
Epoch 50 - Loss: 0.7218
Epoch 60 - Loss: 0.7176
Epoch 70 - Loss: 0.7192
Epoch 80 - Loss: 0.7277
Epoch 90 - Loss: 0.7278
Epoch 100 - Loss: 0.7270
Epoch 110 - Loss: 0.7333
Epoch 120 - Loss: 0.7315
Epoch 130 - Loss: 0.7275
Epoch 140 - Loss: 0.7339
Epoch 150 - Loss: 0.7387
Epoch 160 - Loss: 0.7347
Epoch 170 - Loss: 0.7394
Epoch 180 - Loss: 0.7335
Epoch 190 - Loss: 0.7386
Epoch 200 - Loss: 0.7426
Epoch 210 - Loss: 0.7351
Epoch 220 - Loss: 0.7400
Epoch 230 - Loss: 0.7438
Epoch 240 - Loss: 0.7470
Epoch 250 - Loss: 0.7386
Epoch 260 - Loss: 0.7428
Epoch 270 - Loss: 0.7462
Epoch 280 - Loss: 0.7491
Epoch 290 - Loss: 0.7516
Epoch 300 - Loss: 0.7427
Epoch 310 - Loss: 0.7462
Epoch 320 - Loss: 0.7492
Epoch 330 - Loss: 0.7517
Epoch 340 - Loss: 0.7539
Epoch 350 - Loss: 0.7559
Epoch 360 - Loss: 0.7461
Epoch 370 - Loss: 0.7491
Epoch 380 - Loss: 0.7517
Epoch 390 - Loss: 0.7540
Epoch 400 -

Cod propriu ANN

In [96]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split


def load_images_from_folder(folder, label, size=(32, 32)):
    data = []
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        try:
            img = Image.open(path).convert('RGB').resize(size)
            data.append((np.array(img).flatten() / 255.0, label))
        except:
            continue
    return data

def load_dataset():
    normal = load_images_from_folder('images/normal', 0)
    sepia = load_images_from_folder('images/sepia', 1)
    dataset = normal + sepia
    np.random.shuffle(dataset)
    X, y = zip(*dataset)
    return np.array(X), np.array(y)


class ANN:
    def __init__(self, input_size, hidden_size=64, learning_rate=0.01):
        self.lr = learning_rate
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, 1) * 0.01
        self.b2 = np.zeros((1, 1))

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_deriv(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def forward(self, X):
        self.Z1 = X @ self.W1 + self.b1
        self.A1 = self.sigmoid(self.Z1)
        self.Z2 = self.A1 @ self.W2 + self.b2
        self.A2 = self.sigmoid(self.Z2)
        return self.A2

    def compute_loss(self, y_hat, y):
        y_hat = np.clip(y_hat, 1e-8, 1 - 1e-8)
        return -np.mean(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))

    def backward(self, X, y):
        m = X.shape[0]
        dZ2 = self.A2 - y.reshape(-1, 1)
        dW2 = self.A1.T @ dZ2 / m
        db2 = np.sum(dZ2, axis=0, keepdims=True) / m

        dA1 = dZ2 @ self.W2.T
        dZ1 = dA1 * self.sigmoid_deriv(self.Z1)
        dW1 = X.T @ dZ1 / m
        db1 = np.sum(dZ1, axis=0, keepdims=True) / m

        #actualizare param
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1

    def train(self, X, y, epochs=100):
        for epoch in range(epochs):
            y_hat = self.forward(X)
            loss = self.compute_loss(y_hat, y)
            self.backward(X, y)
            if epoch % 10 == 0 or epoch == epochs - 1:
                print(f"Epoch {epoch} - Loss: {loss:.4f}")

    def predict(self, X):
        y_hat = self.forward(X)
        return (y_hat > 0.5).astype(int).flatten()



X, y = load_dataset()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = ANN(input_size=X.shape[1], hidden_size=64, learning_rate=0.01)
model.train(X_train, y_train, epochs=200)

y_pred = model.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print("Test accuracy:", accuracy)


Epoch 0 - Loss: 0.6932
Epoch 10 - Loss: 0.6932
Epoch 20 - Loss: 0.6932
Epoch 30 - Loss: 0.6932
Epoch 40 - Loss: 0.6932
Epoch 50 - Loss: 0.6932
Epoch 60 - Loss: 0.6932
Epoch 70 - Loss: 0.6932
Epoch 80 - Loss: 0.6932
Epoch 90 - Loss: 0.6932
Epoch 100 - Loss: 0.6932
Epoch 110 - Loss: 0.6932
Epoch 120 - Loss: 0.6932
Epoch 130 - Loss: 0.6932
Epoch 140 - Loss: 0.6932
Epoch 150 - Loss: 0.6932
Epoch 160 - Loss: 0.6932
Epoch 170 - Loss: 0.6932
Epoch 180 - Loss: 0.6932
Epoch 190 - Loss: 0.6932
Epoch 199 - Loss: 0.6932
Test accuracy: 0.5833333333333334


influenta (hyper)parametrilor

In [97]:
def run_experiment(hidden_size=64, learning_rate=0.01, epochs=200):
    X, y = load_dataset()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    model = ANN(input_size=X.shape[1], hidden_size=hidden_size, learning_rate=learning_rate)
    model.train(X_train, y_train, epochs=epochs)
    y_pred = model.predict(X_test)
    accuracy = np.mean(y_pred == y_test)
    print(f"Hidden={hidden_size}, LR={learning_rate}, Epochs={epochs} → Accuracy: {accuracy:.4f}")
    return accuracy


In [98]:
for h in [4, 16, 32, 64, 128]:
    run_experiment(hidden_size=h)



Epoch 0 - Loss: 0.6938
Epoch 10 - Loss: 0.6915
Epoch 20 - Loss: 0.6895
Epoch 30 - Loss: 0.6877
Epoch 40 - Loss: 0.6860
Epoch 50 - Loss: 0.6844
Epoch 60 - Loss: 0.6830
Epoch 70 - Loss: 0.6817
Epoch 80 - Loss: 0.6804
Epoch 90 - Loss: 0.6793
Epoch 100 - Loss: 0.6783
Epoch 110 - Loss: 0.6774
Epoch 120 - Loss: 0.6766
Epoch 130 - Loss: 0.6759
Epoch 140 - Loss: 0.6752
Epoch 150 - Loss: 0.6746
Epoch 160 - Loss: 0.6741
Epoch 170 - Loss: 0.6736
Epoch 180 - Loss: 0.6732
Epoch 190 - Loss: 0.6728
Epoch 199 - Loss: 0.6725
Hidden=4, LR=0.01, Epochs=200 → Accuracy: 0.2500
Epoch 0 - Loss: 0.6946
Epoch 10 - Loss: 0.6919
Epoch 20 - Loss: 0.6899
Epoch 30 - Loss: 0.6883
Epoch 40 - Loss: 0.6870
Epoch 50 - Loss: 0.6860
Epoch 60 - Loss: 0.6852
Epoch 70 - Loss: 0.6847
Epoch 80 - Loss: 0.6842
Epoch 90 - Loss: 0.6838
Epoch 100 - Loss: 0.6836
Epoch 110 - Loss: 0.6834
Epoch 120 - Loss: 0.6832
Epoch 130 - Loss: 0.6831
Epoch 140 - Loss: 0.6831
Epoch 150 - Loss: 0.6830
Epoch 160 - Loss: 0.6830
Epoch 170 - Loss: 0.682

In [99]:
for lr in [0.0001, 0.001, 0.01, 0.1]:
    run_experiment(learning_rate=lr)


Epoch 0 - Loss: 0.6905
Epoch 10 - Loss: 0.6904
Epoch 20 - Loss: 0.6903
Epoch 30 - Loss: 0.6903
Epoch 40 - Loss: 0.6902
Epoch 50 - Loss: 0.6901
Epoch 60 - Loss: 0.6901
Epoch 70 - Loss: 0.6900
Epoch 80 - Loss: 0.6899
Epoch 90 - Loss: 0.6899
Epoch 100 - Loss: 0.6898
Epoch 110 - Loss: 0.6897
Epoch 120 - Loss: 0.6897
Epoch 130 - Loss: 0.6896
Epoch 140 - Loss: 0.6896
Epoch 150 - Loss: 0.6895
Epoch 160 - Loss: 0.6894
Epoch 170 - Loss: 0.6894
Epoch 180 - Loss: 0.6893
Epoch 190 - Loss: 0.6893
Epoch 199 - Loss: 0.6892
Hidden=64, LR=0.0001, Epochs=200 → Accuracy: 0.3333
Epoch 0 - Loss: 0.6931
Epoch 10 - Loss: 0.6929
Epoch 20 - Loss: 0.6927
Epoch 30 - Loss: 0.6925
Epoch 40 - Loss: 0.6923
Epoch 50 - Loss: 0.6922
Epoch 60 - Loss: 0.6920
Epoch 70 - Loss: 0.6919
Epoch 80 - Loss: 0.6918
Epoch 90 - Loss: 0.6917
Epoch 100 - Loss: 0.6916
Epoch 110 - Loss: 0.6915
Epoch 120 - Loss: 0.6914
Epoch 130 - Loss: 0.6913
Epoch 140 - Loss: 0.6913
Epoch 150 - Loss: 0.6912
Epoch 160 - Loss: 0.6912
Epoch 170 - Loss: 0.

In [100]:
for e in [50, 100, 200, 500]:
    run_experiment(epochs=e)

Epoch 0 - Loss: 0.6922
Epoch 10 - Loss: 0.6869
Epoch 20 - Loss: 0.6846
Epoch 30 - Loss: 0.6837
Epoch 40 - Loss: 0.6833
Epoch 49 - Loss: 0.6831
Hidden=64, LR=0.01, Epochs=50 → Accuracy: 0.3333
Epoch 0 - Loss: 0.6937
Epoch 10 - Loss: 0.6919
Epoch 20 - Loss: 0.6912
Epoch 30 - Loss: 0.6909
Epoch 40 - Loss: 0.6907
Epoch 50 - Loss: 0.6906
Epoch 60 - Loss: 0.6906
Epoch 70 - Loss: 0.6906
Epoch 80 - Loss: 0.6906
Epoch 90 - Loss: 0.6906
Epoch 99 - Loss: 0.6906
Hidden=64, LR=0.01, Epochs=100 → Accuracy: 0.4167
Epoch 0 - Loss: 0.6940
Epoch 10 - Loss: 0.6920
Epoch 20 - Loss: 0.6912
Epoch 30 - Loss: 0.6908
Epoch 40 - Loss: 0.6907
Epoch 50 - Loss: 0.6906
Epoch 60 - Loss: 0.6906
Epoch 70 - Loss: 0.6906
Epoch 80 - Loss: 0.6906
Epoch 90 - Loss: 0.6906
Epoch 100 - Loss: 0.6906
Epoch 110 - Loss: 0.6906
Epoch 120 - Loss: 0.6906
Epoch 130 - Loss: 0.6906
Epoch 140 - Loss: 0.6906
Epoch 150 - Loss: 0.6906
Epoch 160 - Loss: 0.6906
Epoch 170 - Loss: 0.6906
Epoch 180 - Loss: 0.6906
Epoch 190 - Loss: 0.6906
Epoch 

CNN cod propriu

In [139]:
import numpy as np
from PIL import Image
import os

def load_images_from_folder(folder, label, size=(32, 32)):
    data = []
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        try:
            img = Image.open(path).convert('RGB').resize(size)
            data.append((np.array(img) / 255.0, label))
        except:
            continue
    return data

def load_dataset():
    data = load_images_from_folder("images/normal", 0) + load_images_from_folder("images/sepia", 1)
    np.random.shuffle(data)
    X, y = zip(*data)
    return np.array(X), np.array(y)


In [140]:
def convolve2d(img, kernel):
    h, w = img.shape
    kh, kw = kernel.shape
    out = np.zeros((h - kh + 1, w - kw + 1))
    for i in range(out.shape[0]):
        for j in range(out.shape[1]):
            out[i, j] = np.sum(img[i:i+kh, j:j+kw] * kernel)
    return out


In [141]:
class SimpleCNN:
    def __init__(self):
        # Kerneluri random (3x3) pentru fiecare canal
        self.kernel_r = np.random.randn(3, 3)
        self.kernel_g = np.random.randn(3, 3)
        self.kernel_b = np.random.randn(3, 3)
        self.W = np.random.randn(30*30, 1) * 0.01  # After conv and flatten, adjusted size to 30x30
        self.b = np.zeros((1,))

    def relu(self, x):
        return np.maximum(0, x)

    def forward(self, X):
        conv_r = np.array([convolve2d(img[:, :, 0], self.kernel_r) for img in X])
        conv_g = np.array([convolve2d(img[:, :, 1], self.kernel_g) for img in X])
        conv_b = np.array([convolve2d(img[:, :, 2], self.kernel_b) for img in X])
        conv_total = self.relu(conv_r + conv_g + conv_b)  # Shape: (n_samples, 30, 30)

        flat = conv_total.reshape(len(X), -1)  # Flatten (n_samples, 900) # Adjusted to (n_samples, 30x30=900)
        z = flat @ self.W + self.b
        return 1 / (1 + np.exp(-z))  # sigmoid

    def compute_loss(self, y_hat, y):
        y_hat = np.clip(y_hat, 1e-8, 1 - 1e-8)
        return -np.mean(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))

    def backward(self, X, y, y_hat, learning_rate=0.01):
        m = y.shape[0]
        dz = y_hat - y.reshape(-1, 1)
        conv_r = np.array([convolve2d(img[:, :, 0], self.kernel_r) for img in X])
        conv_g = np.array([convolve2d(img[:, :, 1], self.kernel_g) for img in X])
        conv_b = np.array([convolve2d(img[:, :, 2], self.kernel_b) for img in X])
        conv_total = self.relu(conv_r + conv_g + conv_b)
        flat = conv_total.reshape(m, -1)

        dW = flat.T @ dz / m
        db = np.sum(dz) / m

        self.W -= learning_rate * dW
        self.b -= learning_rate * db

    def train(self, X, y, epochs=50, learning_rate=0.01):
        for epoch in range(epochs):
            y_hat = self.forward(X)
            loss = self.compute_loss(y_hat, y)
            self.backward(X, y, y_hat, learning_rate)
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        y_hat = self.forward(X)
        return (y_hat > 0.5).astype(int).flatten()

In [142]:
from sklearn.model_selection import train_test_split

X, y = load_dataset()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

cnn = SimpleCNN()
cnn.train(X_train, y_train, epochs=100, learning_rate=0.01)

y_pred = cnn.predict(X_test)
acc = np.mean(y_pred == y_test)
print(f"Test Accuracy: {acc:.4f}")


Epoch 0, Loss: 0.7744
Epoch 10, Loss: 3.0577
Epoch 20, Loss: 7.4834
Epoch 30, Loss: 2.6251
Epoch 40, Loss: 2.3224
Epoch 50, Loss: 8.7485
Epoch 60, Loss: 3.9691
Epoch 70, Loss: 4.7558
Epoch 80, Loss: 7.7348
Epoch 90, Loss: 3.7317
Test Accuracy: 0.8750
